In [15]:
from typing import Iterable
from sampo.schemas import Time
from sampo.scheduler.multi_agency.block_generator import generate_blocks, SyntheticBlockGraphType
from sampo.scheduler.multi_agency.multi_agency import StochasticManager, ScheduledBlock
from sampo.scheduler.multi_agency import Agent, validate_block_schedule
from uuid import uuid4
from copy import deepcopy

from sampo.generator import SimpleSynthetic
from sampo.scheduler import HEFTScheduler
from random import Random

r_seed = 231
rand = Random(r_seed)
ss = SimpleSynthetic(rand)

scheduler_constructors = [HEFTScheduler, HEFTScheduler, HEFTScheduler, HEFTScheduler, HEFTScheduler, HEFTScheduler, HEFTScheduler, HEFTScheduler]
base_contractor = ss.contractor(10)

bg = generate_blocks(SyntheticBlockGraphType.SEQUENTIAL, 100, [1, 1, 1], lambda x: (50, 60), 0.5, rand)

def finish_time(schedule: Iterable[ScheduledBlock]) -> Time:
    return max([sblock.end_time for sblock in schedule])

SEQUENTIAL PARALLEL PARALLEL PARALLEL PARALLEL GENERAL GENERAL SEQUENTIAL GENERAL SEQUENTIAL GENERAL GENERAL SEQUENTIAL GENERAL GENERAL SEQUENTIAL PARALLEL SEQUENTIAL PARALLEL SEQUENTIAL GENERAL PARALLEL SEQUENTIAL SEQUENTIAL GENERAL SEQUENTIAL PARALLEL PARALLEL PARALLEL GENERAL GENERAL SEQUENTIAL SEQUENTIAL PARALLEL PARALLEL SEQUENTIAL PARALLEL SEQUENTIAL GENERAL PARALLEL PARALLEL GENERAL SEQUENTIAL SEQUENTIAL GENERAL SEQUENTIAL SEQUENTIAL SEQUENTIAL SEQUENTIAL GENERAL GENERAL GENERAL PARALLEL GENERAL GENERAL GENERAL PARALLEL GENERAL PARALLEL GENERAL PARALLEL PARALLEL GENERAL PARALLEL PARALLEL SEQUENTIAL PARALLEL GENERAL SEQUENTIAL SEQUENTIAL PARALLEL SEQUENTIAL GENERAL GENERAL PARALLEL PARALLEL PARALLEL GENERAL PARALLEL GENERAL SEQUENTIAL GENERAL GENERAL SEQUENTIAL PARALLEL PARALLEL GENERAL PARALLEL SEQUENTIAL PARALLEL SEQUENTIAL GENERAL PARALLEL PARALLEL SEQUENTIAL GENERAL PARALLEL GENERAL GENERAL SEQUENTIAL GENERAL


In [16]:
from sampo.schemas import WorkerProductivityMode, Contractor, IntervalGaussian
from sampo.schemas.time_estimator import DefaultWorkEstimator
from sampo.scheduler.multi_agency import Manager
from typing import Callable


def construct_agent(i: int, scheduler_constructor, contractor: Contractor) -> Agent:
    work_estimator = DefaultWorkEstimator()
    work_estimator.set_productivity_mode(WorkerProductivityMode.Stochastic)
    for worker in ['driver', 'fitter', 'manager', 'handyman', 'electrician', 'engineer']:
        work_estimator.set_worker_productivity(worker, IntervalGaussian(1, 0.1 * i, 0, 2))
    
    return Agent(f'Agent {i}', scheduler_constructor(work_estimator=work_estimator), [contractor])

def test_with_manager(manager_constructor: Callable[[list[Agent]], Manager]):
    contractors = [deepcopy(base_contractor) for _ in scheduler_constructors]
    for contractor in contractors:
        contractor.id = str(uuid4())
    
    agents = [construct_agent(i, scheduler_constructors[i % len(scheduler_constructors)], contractor)
              for i, contractor in enumerate(contractors)]
    manager = manager_constructor(agents)
    
    scheduled_blocks = manager.manage_blocks(bg)
    # validate_block_schedule(bg, scheduled_blocks, agents)
    
    return finish_time(scheduled_blocks.values())

In [17]:
time_static = test_with_manager(Manager)
time_static

16083

In [18]:
time_stochastic = test_with_manager(StochasticManager)
time_stochastic

16185

In [19]:
def run_comparison(iterations: int):
    successes = 0
    for i in range(iterations):
        time_static = test_with_manager(Manager)
        time_stochastic = test_with_manager(StochasticManager)
        print(f'{time_static} {time_stochastic}')
        if time_stochastic < time_static:
            successes += 1
    print(f'Success percentage: {int(100 * successes / iterations)}% on {iterations} iterations')

run_comparison(10)

15794 16185
15715 16185
16488 16185
15887 16185
15801 16185
16403 16185
16656 16185
15943 16185
16399 16185
16231 16185
Success percentage: 50% on 10 iterations
